# 🥋 Lekcja 9: Requires Grad & Inference Mode (Zarządzanie Pamięcią)

Każdy tensor w PyTorch ma flagę `requires_grad`.
*   `True`: PyTorch alokuje dodatkową pamięć na graf obliczeniowy.
*   `False`: Tensor zachowuje się jak zwykła macierz NumPy (lekki).

Podczas **treningu** chcemy `True` (dla wag).
Podczas **używania (Inference)** chcemy `False` (żeby nie zapchać pamięci).

Mamy trzy sposoby na kontrolowanie tego:
1.  **.detach():** Fizyczne oderwanie tensora od grafu.
2.  **with torch.no_grad():** Klasyczny sposób na wyłączenie silnika Autograd.
3.  **with torch.inference_mode():** Nowoczesny, ekstremalnie zoptymalizowany tryb dla produkcji.

In [1]:
import torch

# 1. Tensor, który chce się uczyć (Wagi)
w = torch.randn(5, 5, requires_grad=True)

# 2. Tensor, który jest danymi (Input)
x = torch.randn(5, 5) # Domyślnie requires_grad=False

print(f"Wagi (w) requires_grad: {w.requires_grad}")
print(f"Dane (x) requires_grad: {x.requires_grad}")

# Operacja
y = w @ x

# Wynik "dziedziczy" chęć uczenia się po rodzicach!
# Skoro 'w' wymagało gradientu, to 'y' też go wymaga (żeby policzyć pochodną dla 'w').
print(f"Wynik (y) requires_grad: {y.requires_grad}")
print(f"Funkcja y: {y.grad_fn}")

Wagi (w) requires_grad: True
Dane (x) requires_grad: False
Wynik (y) requires_grad: True
Funkcja y: <MmBackward0 object at 0x0000014F58561780>


## 1. Metoda `.detach()` (Chirurgiczne cięcie)

Używamy tego, gdy chcemy przerwać przepływ gradientów w połowie sieci.
Częsty przypadek: **GAN-y** lub **Transfer Learning** (zamrażanie części sieci).

`.detach()` zwraca nowy tensor, który dzieli tę samą pamięć, ale **nie ma historii**.

In [2]:
# Mamy y, które jest częścią grafu
print(f"Oryginał y: {y.grad_fn} (Ma historię)")

# Odcinamy
z = y.detach()

print(f"Odcięty z:  {z.grad_fn} (Brak historii)")
print(f"Czy z wymaga gradientu? {z.requires_grad}")

# Dowód na współdzielenie pamięci (Uważaj na in-place!)
print("\n--- TEST PAMIĘCI ---")
z[0, 0] = 999
print(f"Zmieniliśmy z[0,0]. Sprawdźmy y[0,0]: {y[0,0]}")
print("Uwaga: .detach() nie kopiuje danych! Zmiana w 'z' zmienia 'y'.")

Oryginał y: <MmBackward0 object at 0x0000014F58562E90> (Ma historię)
Odcięty z:  None (Brak historii)
Czy z wymaga gradientu? False

--- TEST PAMIĘCI ---
Zmieniliśmy z[0,0]. Sprawdźmy y[0,0]: 999.0
Uwaga: .detach() nie kopiuje danych! Zmiana w 'z' zmienia 'y'.


## 2. `no_grad()` vs `inference_mode()`

To są menedżery kontekstu (`with ...`).

*   **`torch.no_grad()`**: Tylko ustawia `requires_grad=False` dla nowych tensorów. Ale nadal pozwala na pewne operacje, które mogą być potrzebne do `backward` w przyszłości.
*   **`torch.inference_mode()` (ZALECANE)**: Wyłącza wszystko. Nie można użyć wyniku z tego bloku do żadnego treningu. Dzięki temu PyTorch może pominąć np. śledzenie wersji (Version Counter), co przyspiesza kod.

In [3]:
print("--- NO GRAD ---")
with torch.no_grad():
    y_no_grad = w @ x
    print(f"Czy wymaga gradientu? {y_no_grad.requires_grad}")
    # W no_grad() wciąż możemy robić in-place operations, które są śledzone przez Version Counter
    
print("\n--- INFERENCE MODE (Black Belt Choice) ---")
with torch.inference_mode():
    y_inf = w @ x
    print(f"Czy wymaga gradientu? {y_inf.requires_grad}")
    
    # Różnica jest subtelna, ale kluczowa.
    # Spróbujmy użyć tych wyników do dalszego treningu poza blokiem.
    
try:
    # To zadziała (choć nie policzy gradientu dla w, bo y_no_grad nie ma historii)
    loss = y_no_grad.sum()
    loss.backward() 
    print("Backward na no_grad: Przeszło (ale gradientów brak).")
except Exception as e:
    print(f"Błąd no_grad: {e}")

try:
    # To wyrzuci błąd! Inference Mode zabrania tworzenia grafu nawet później.
    loss = y_inf.sum()
    loss.backward()
except RuntimeError as e:
    print(f"\n🚫 Backward na inference_mode: ZABLOKOWANE.")
    print(f"Błąd: {e}")

--- NO GRAD ---
Czy wymaga gradientu? False

--- INFERENCE MODE (Black Belt Choice) ---
Czy wymaga gradientu? False
Błąd no_grad: element 0 of tensors does not require grad and does not have a grad_fn

🚫 Backward na inference_mode: ZABLOKOWANE.
Błąd: element 0 of tensors does not require grad and does not have a grad_fn


## Dlaczego `inference_mode` jest szybsze?

W `inference_mode` PyTorch pomija tworzenie struktur `ViewTracking` i `VersionCounter`.
Przy małych modelach (MLP) różnica jest znikoma.
Przy gigantycznych modelach (LLM, ViT) i bardzo małych batchach (np. obsługa zapytań HTTP w czasie rzeczywistym), narzut Pythona na obsługę grafu może być zauważalny.

**Zasada inżynierska:**
*   Trenujesz? -> Nic nie rób.
*   Walidujesz/Testujesz/Wdrażasz? -> **`@torch.inference_mode()`** (jako dekorator funkcji).

In [4]:
# Wzorzec projektowy: Dekorator
@torch.inference_mode()
def predict(model_weights, inputs):
    # Cała ta funkcja jest chroniona i zoptymalizowana
    return model_weights @ inputs

# Symulacja "Modelu" na produkcji
result = predict(w, x)

print("Wynik predykcji:", result.shape)
print("Czy ma historię?", result.grad_fn)

Wynik predykcji: torch.Size([5, 5])
Czy ma historię? None


## 🥋 Black Belt Summary

1.  **`x.requires_grad`**: Własność wirusowa. Jeśli jeden składnik równania tego wymaga, wynik też będzie tego wymagał.
2.  **`.detach()`**: Używaj, gdy chcesz "urwać" historię (np. przekazując stan ukryty w LSTM do nowej sekwencji albo wizualizując tensor w matplotlib).
3.  **`torch.no_grad()`**: Stare, dobre, ale wolniejsze. Używaj tylko, jeśli musisz manipulować tensorami w specyficzny sposób.
4.  **`torch.inference_mode()`**: **Nowy standard.** Używaj zawsze na produkcji i podczas walidacji. Jest szybsze i bezpieczniejsze (gwarantuje, że nie zrobisz błędu w logice gradientów).